In [55]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import random
import spacy
import pytextrank
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer,RobertaTokenizer, LongformerTokenizer,  BigBirdTokenizerFast
from transformers import BertForSequenceClassification, RobertaForSequenceClassification, LongformerForSequenceClassification, BigBirdForSequenceClassification

In [10]:
# If there's a GPU available...
# If GPU not available, training will cost SEVERAL DAYS, not recommended running on CPU
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU insteadp(not recommended).')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1070


In [2]:
#Load the model, if no model to load, initialize all the model and tokenizer for further use.
#There will be several warning message pop out when initializing the model, 
# it is because of the model haven't been trained, it could be ignored.

checkpoint_path = "Model.pth"  # Change to your preferred location

if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    losses = checkpoint['losses']
    print("Loaded checkpoint.")
else:
    tokenizer_options = {
        "bert": BertTokenizer.from_pretrained('bert-base-uncased'),
        "roberta": RobertaTokenizer.from_pretrained('roberta-base'),
        "longformer": LongformerTokenizer.from_pretrained('allenai/longformer-base-4096'),
        "bigbird":  BigBirdTokenizerFast.from_pretrained('l-yohai/bigbird-roberta-base-mnli')
    }

    model_options = {
        "bert": BertForSequenceClassification.from_pretrained('bert-base-uncased'),
        "roberta": RobertaForSequenceClassification.from_pretrained('roberta-base'),
        "longformer": LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096'),
        "bigbird": BigBirdForSequenceClassification.from_pretrained('l-yohai/bigbird-roberta-base-mnli')
    }
    

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Set all the variable here:

In [56]:
# Summarize the input to how many tokens, default to 512 for BERT use.
# Since LongFormer could accept 4096 tokens, we may could skip TextRank if LongFormer
ENABLE_TEXT_RANK = False
TEXT_RANK_LENGTH = 512

MODEL = "bert" # Choose from "bert", "roberta", "longformer", "bigbird"
seed_val = 42

TRAIN_DATA_PATH = "data/ds1_train.csv"
TEST_DATA_PATH = "data/ds1_test.csv"
DATASET = "ds1" if "ds1" in TRAIN_DATA_PATH else "ds2"
LEARNING_RATE = 2e-5
NUM_EPOCHS = 4

In [29]:
# Use the selected tokenizer and model
tokenizer = tokenizer_options.get(MODEL)
model = model_options.get(MODEL)

# Set the seed value all over the place to make this reproducible.
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [33]:
train = pd.read_csv(TRAIN_DATA_PATH)
test = pd.read_csv(TEST_DATA_PATH)

In [34]:
print(train.shape)
train.head()

(2120, 3)


,label,text,label_id
0,fantasy,"in the early days of the conquest, when the r...",0
1,fantasy,long before she was the terror of wonderland t...,0
2,fantasy,we see you. now we are you. no real witch woul...,0
3,fantasy,arch swindler moist van lipwig never believed ...,0
4,fantasy,tithe follows the story of sixteen year old a...,0


By now, data has been cleaned. Next step is to tokenize, train, and test.
The first step is to use TextRank from Gensim to rank the first few sentences, and then we will train on BERT and RoBERTA to see its' performance.

# Text Rank by Spacy:
Note that this could be skipped.

In [44]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank")

def summarize(input_text: str, max_tokens: int = 512):
    # Load a SpaCy model. For example, the English model.


    # Parse the document with SpaCy.
    doc = nlp(input_text)

    # If the text is less than 512 words, return the original text.
    if len(doc) <= max_tokens:
        return input_text

    # Extract the phrases using TextRank.
    phrases = ", ".join([p.text for p in doc._.phrases])

    summarized_doc = []  # Initialize an empty list to store the summarized sentences

    # Iterate over the sentences generated by textrank.summary and append them to the list
    for sent in doc._.textrank.summary(limit_sentences=50):
        summarized_doc.append(sent)


    # Join the sentences to form the summary.
    summary = " ".join([span.text for span in summarized_doc])

    # If the summary is longer than max_tokens, truncate it.
    if len(summary.split()) > max_tokens:
        summary = " ".join(summary.split()[:max_tokens])

    return summary

In [48]:
if (ENABLE_TEXT_RANK):
    print("\n-----------------------Progressing---------------------\n")
    try:
        train["text"] = train["text"].apply(summarize)
        test["text"] = test["text"].apply(summarize)
    except Exception as e:
        print("An error occurred while applying text_rank:", str(e))
    print("\n-----------------------Done---------------------\n")
else:
    print('TextRank has been skipped')

TextRank has been skipped


# Tokenize the data

In [42]:
train_text = train.text.values
train_labels = train.label_id.values

test_text = test.text.values
test_labels = test.label_id.values

In [43]:
# Tokenize the text
def tokenize_text(summary, max_length=512):
    # Tokenize the 'summary' column of the DataFrame using the tokenizer's batch_encode_plus method
    # df['summary'].tolist() converts the 'summary' column into a list of strings
    # max_length specifies the maximum length of the tokenized sequences
    # padding='max_length' pads the tokenized sequences to have a length of max_length
    # truncation=True truncates the tokenized sequences if they exceed the max_length
    return tokenizer.batch_encode_plus(
        summary.tolist(), 
        max_length=max_length, 
        padding='max_length', 
        truncation=True
    )

tokenized_train_texts = tokenize_text(train_summary)
tokenized_test_texts = tokenize_text(test_summary)

In [54]:
class TextClassificationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create the dataset
train_dataset = TextClassificationDataset(tokenized_train_texts, train['label_id'].values)
test_dataset = TextClassificationDataset(tokenized_test_texts, test['label_id'].values)

# Create the dataloader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

# train the model

In [ ]:
# Initialize the optimizer
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

# Move the model to the GPU
model.train()
model.to('cuda')

# Initialize a list to store the loss values
losses = []

checkpoint_path = f"{MODEL}_{DATASET}_TextRank_{ENABLE_TEXT_RANK}.pth" # Change to your preferred location

start_epoch = 0

# Load from checkpoint if it exists
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    losses = checkpoint['losses']
    print("Loaded checkpoint.")

# Training 
for epoch in range(start_epoch, NUM_EPOCHS):
    print(f"Starting epoch {epoch}")
    for batch in tqdm(train_dataloader, desc="Training"):
        batch = {k: v.to('cuda') for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        # Optimize the model parameters
        optimizer.step()
        optimizer.zero_grad()

        # Append the loss value to the list
        losses.append(loss.item())  

    # Save checkpoint after each epoch
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'losses': losses,
    }, checkpoint_path)
    print(f"Saved checkpoint for epoch {epoch}.")

    # Plot the loss values after each epoch
    plt.plot(losses)
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.title('Loss over Iterations')
    plt.show()


# Model now been trained, now evaluate the performance

In [ ]:
#If yo have a trained model just want to evaluate it:
checkpoint_path = "Model.pth"  # Change to your preferred location
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    losses = checkpoint['losses']
    print("Loaded checkpoint.")

In [ ]:
model.eval()
model.to('cuda')
predictions = []
true_labels = []

# Iterate over the test dataset
for batch in test_dataloader:
    batch = {k: v.to('cuda') for k, v in batch.items()}

    # Disable gradient calculation
    with torch.no_grad():
        # Perform inference
        outputs = model(**batch)
        
    predicted_values = outputs.logits
    predictions.extend(predicted_values)
    true_labels.extend(batch['labels'].tolist())

# Convert logits to predictions
predictions = [torch.argmax(item).item() for item in predictions]

# Make sure that true_labels is a list
assert isinstance(true_labels, list)

from sklearn.metrics import classification_report

  
# Generate classification report
report = classification_report(true_labels, predictions)

print(report)

# Save the report to a file
with open(f'Classreport_{MODEL}_{DATASET}_TextRank_{ENABLE_TEXT_RANK}.txt', 'w') as file:
    file.write(report)
